In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from torch.autograd import Variable

from data_load_gan import index_group, data_loading
from CNN_models import ConvNet_2


In [ ]:
# training code 
# the path to training data and test data
path_train_data = '../../Sync/Kun/30_min_ele/non_ele_whole/train/'
path_test_data = '../../Sync/Kun/30_min_ele/non_ele_whole/test/'
# the path to save trained models
path_save = '../../Sync/Kun/0718CNN_Res/'
# number of data
train_num = 10000
test_num = 2000
# training details to be defined
batch_size = 196
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
epoches = 8000
learning_rate = 0.001

# load the deep learning model
model = ConvNet_2()
model = model.cuda()
model = nn.DataParallel(model).cuda()

# loss function and optimizer
loss_function = nn.L1Loss()
loss_function = loss_function.cuda()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 1e-6)
input_noise = torch.FloatTensor(batch_size, 5, 100, 100)
input_noise = input_noise.cuda()
input_noise = Variable(input_noise)
costs = []
test_costs = []
print_cost = True

# schedule for the learning rate
lambda1 = lambda epoch: 1/np.sqrt(((epoch % 2000)+1.0))
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda1)

# training process
for epoch in range(epoches):
    print(epoch)
    epoch_cost = 0
    test_loss = 0
    # form the group of training data
    train_group, test_group = index_group(batch_size, train_num, test_num)
    # start to train by batch
    for batch_id in train_group:
        
        batch_inp, batch_op = data_loading(batch_id, path_train_data)
        batch_inp = torch.tensor(batch_inp, dtype = torch.float)
        batch_op = torch.tensor(batch_op, dtype = torch.float)
        
        optimizer.zero_grad()

        batch_inp = batch_inp.cuda()
        batch_op = batch_op.cuda()
        
        input_noise.normal_(0, 0.02)
        batch_inp.add_(input_noise) 
        
        prediction = model.module.forward(batch_inp)
        #print(prediction.shape)
        loss = loss_function(prediction, batch_op)   
        loss.backward()
        optimizer.step()    
        epoch_cost = epoch_cost + loss.item()
    
    # test data to check the training performance
    with torch.no_grad():
        test_inp, test_op = data_loading(test_group, path_test_data)
        test_inp = torch.tensor(test_inp, dtype = torch.float)
        test_op = torch.tensor(test_op, dtype = torch.float)
        test_inp = test_inp.cuda()
        test_op = test_op.cuda()
        test_prediction = model.module.forward(test_inp)
        temp_test_loss = loss_function(test_prediction, test_op)
        test_loss = test_loss + temp_test_loss.item()
    
    # print the training cost and test cost after every several steps
    if print_cost == True and epoch % 10 == 0:
        print ("Cost after epoch %i: %f" % (epoch+1, epoch_cost))
        print ("Cost of test after epoch %i: %f" % (epoch+1, test_loss))
        
        test_prediction = test_prediction.cpu()
        test_prediction = np.array(test_prediction)
        save_fig(test_prediction[0,0,:,:], path_save + str(epoch) + '_prediction.jpg')
        
        test_op = test_op.cpu()
        test_op = np.array(test_op)
        save_fig(test_op[0,0,:,:], path_save + str(epoch) + '_label.jpg')
    
    if print_cost == True and epoch % 5 == 0:
        costs.append(epoch_cost)
        test_costs.append(test_loss)
    # save trained model/parameters after several steps
    if epoch >= 50 and epoch % 100 == 0:
        torch.save(model, path_save + str(epoch) + '_model')
    